<a href="https://colab.research.google.com/github/Shwethaa-R/Deep-Learning-Activity-6/blob/main/AutoEncoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AutoEncoders

The autoencoder implemented here is a Stacked Autoencoder. A stacked autoencoder is a neural network consist several layers of sparse autoencoders where output of each hidden layer is connected to the input of the successive hidden layer. A Sparse Autoencoder is a type of autoencoder that employs sparsity to achieve an information bottleneck. Specifically the loss function is constructed so that activations are penalized within a layer.

##Downloading the dataset

The autoencoder built in the following section is used for building a recommender system, which gives rating for movies. The dataset used for the same is a movie rating dataset. There are two files, namely ml-100k (contains 100k entries) and ml-1m (contains 1m entries). PyTorch is used for the implementation of the autoencoder.

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-05-18 14:21:01--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  26.2MB/s    in 0.2s    

2021-05-18 14:21:01 (26.2 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2021-05-18 14:21:05--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  33.1MB/s    in 0.2s    

2021-05-18 14:21:06 (33.1 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [4]:
# We won't be using this dataset. This dataset is to give a basic idea of the parameters.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [5]:
# This dataset contains the number assigned to the movie, name of the movie along with its genre respectively.
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# This dataset contains the number assigned to the user, sex of the user, their age and the pincode respectively. 
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
# This dataset contains the number assigned to the user, the movie number, the rating and the timestamp respectively.
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and the test set


In [8]:
# Training data = 80%, Testing data = 20%
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
# Converting the dataframe into a numpy array
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
# Converting the dataframe into a numpy array
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


In [9]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [10]:
# Torch tensors only accept list of lists, and not dataframes as input
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [11]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [12]:
# Autoencoders are a combination of single layer feed forward networks and hence nn.Linear() is used. The input layer has the same number of neurons as the number of movies, 
# which is the number of neurons in the output layer as well. The activation function used is sigmoid. Optimizer used is RMSProp.  
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    # Forward propagation using sigmoid activation function except on the final layer as it has to calculate the rating
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE


In [13]:
# The model is trained for 200 epochs
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    # Torch doesn't accept a single vector of one dimension. It should be in the form of a batch.
    input = Variable(training_set[id_user]).unsqueeze(0)
    # Target values are the same as the input values.
    target = input.clone()
    # Processing only those users' ratings who have rated at least one movie
    if torch.sum(target.data > 0) > 0:
      # The predicted ratings by the model
      output = sae(input)
      target.require_grad = False
      # Setting the predicted values to 0 for the unrated movies
      output[target == 0] = 0
      # Computing the loss for the current user ratings between the predicted and actual values
      loss = criterion(output, target)
      # Term used for updating the loss function
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      # Backward propagation
      loss.backward()
      # Computing the total loss until the current epoch
      train_loss += np.sqrt(loss.data*mean_corrector)
      # Calculating the total number of users who have rated at least one movie
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'    loss: '+ str(train_loss/s))

epoch: 1    loss: tensor(1.7711)
epoch: 2    loss: tensor(1.0967)
epoch: 3    loss: tensor(1.0532)
epoch: 4    loss: tensor(1.0383)
epoch: 5    loss: tensor(1.0307)
epoch: 6    loss: tensor(1.0265)
epoch: 7    loss: tensor(1.0239)
epoch: 8    loss: tensor(1.0219)
epoch: 9    loss: tensor(1.0208)
epoch: 10    loss: tensor(1.0196)
epoch: 11    loss: tensor(1.0190)
epoch: 12    loss: tensor(1.0183)
epoch: 13    loss: tensor(1.0182)
epoch: 14    loss: tensor(1.0178)
epoch: 15    loss: tensor(1.0174)
epoch: 16    loss: tensor(1.0170)
epoch: 17    loss: tensor(1.0168)
epoch: 18    loss: tensor(1.0164)
epoch: 19    loss: tensor(1.0167)
epoch: 20    loss: tensor(1.0160)
epoch: 21    loss: tensor(1.0160)
epoch: 22    loss: tensor(1.0160)
epoch: 23    loss: tensor(1.0160)
epoch: 24    loss: tensor(1.0157)
epoch: 25    loss: tensor(1.0158)
epoch: 26    loss: tensor(1.0155)
epoch: 27    loss: tensor(1.0152)
epoch: 28    loss: tensor(1.0153)
epoch: 29    loss: tensor(1.0136)
epoch: 30    loss: tens

## Testing the SAE


In [14]:
# Performing the same procedure as done on the training data, on the test data to compute the test loss
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9572)
